In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import ttest_ind

### 3.1

In [2]:
df_data = pd.read_excel('df.xlsx', sheet_name='Dataset')
df_city = pd.read_excel('df.xlsx', sheet_name='city_dict')

In [3]:
df_data.head()

,id_view,time_view,id_group,id_city,nflag_order
0,10457162393,2023-01-17 13:56:06.118,0,3,1
1,10457152884,2023-01-30 17:38:18.629,1,1,1
2,10457162947,2023-01-07 14:06:22.689,1,1,1
3,10457197478,2023-01-02 11:02:58.189,0,3,0
4,10457176480,2023-01-04 22:56:19.240,0,1,1


In [4]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21939 entries, 0 to 21938
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_view      21939 non-null  int64         
 1   time_view    21939 non-null  datetime64[ns]
 2   id_group     21939 non-null  int64         
 3   id_city      21939 non-null  int64         
 4   nflag_order  21939 non-null  int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 857.1 KB


In [5]:
df_city.head()

,id_city,name_city
0,1,Москва
1,2,Санкт-Петербург
2,3,Новосибирск
3,4,Казань
4,5,Краснодар


In [6]:
df_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id_city    11 non-null     int64 
 1   name_city  11 non-null     object
dtypes: int64(1), object(1)
memory usage: 308.0+ bytes


In [7]:
df = df_data.merge(df_city, on='id_city')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21939 entries, 0 to 21938
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_view      21939 non-null  int64         
 1   time_view    21939 non-null  datetime64[ns]
 2   id_group     21939 non-null  int64         
 3   id_city      21939 non-null  int64         
 4   nflag_order  21939 non-null  int64         
 5   name_city    21939 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 1.2+ MB


### 3.2

In [9]:
def test_calc(r1, r2, alpha = 0.05):
    
    s,p = ttest_ind(r1,r2)
    
    if p < alpha:
        print("Есть статистически значимые различия между группами.")
    else:
        print("Статистически значимых различий между группами нет.")
    
    print("Среднее значение 1 ряда", r1.mean())
    print("Среднее значение 2 ряда", r2.mean())
    print("Разница средних = ", r1.mean()-r2.mean())
    print("P_value = ",p)
    return s, p

In [10]:
group_a = df[df['id_group'] == 0]['nflag_order']  # Старая цена
group_b = df[df['id_group'] == 1]['nflag_order']  # Новая цена

In [11]:
s,p = test_calc(group_a,group_b)

Есть статистически значимые различия между группами.
Среднее значение 1 ряда 0.6150932939220395
Среднее значение 2 ряда 0.6757851165301899
Разница средних =  -0.060691822608150425
P_value =  5.1365957325220535e-21


На основе этих данных можно сделать вывод, что введение новой платежной механики привело к изменению поведения пользователей в положительную сторону. Однако при принятии решения о внедрении новой ценовой политики важно учитывать не только увеличение конверсии, но и другие бизнес-показатели, такие как общий доход, удовлетворенность клиентов и долгосрочная лояльность.

### 3.3

In [12]:
# Создание списка уникальных городов
un_city = df['name_city'].unique()

In [13]:
un_city

array(['Новосибирск', 'Москва', 'Санкт-Петербург', 'Казань',
       'Архангельск', 'Челябинск', 'Сочи', 'Хабаровск', 'Нижний Новгород',
       'Краснодар', 'Воронеж'], dtype=object)

In [14]:
# Инициализация списка для хранения результатов
stat_city = []

In [15]:
stat_city

[]

In [16]:
for city in un_city:
    g_a = df.loc[(df['id_group'] == 0) & (df['name_city'] == city), 'nflag_order']
    g_b = df.loc[(df['id_group'] == 1) & (df['name_city'] == city), 'nflag_order']
    if len(g_a) > 0 and len(g_b) > 0: # Проверяем, что в обеих группах достаточно данных для t-теста
        print(city)
        s_t, p_t = test_calc(g_a, g_b) 
        if p_t < 0.05:
            stat_city.append(city)
        print()
print('Статистически значимая разница присутствует в городах:', stat_city)


Новосибирск
Статистически значимых различий между группами нет.
Среднее значение 1 ряда 0.6283405842137973
Среднее значение 2 ряда 0.6531585220500596
Разница средних =  -0.024817937836262294
P_value =  0.13822636677161423

Москва
Есть статистически значимые различия между группами.
Среднее значение 1 ряда 0.6172531214528945
Среднее значение 2 ряда 0.691944194419442
Разница средних =  -0.07469107296654753
P_value =  1.368947706664987e-13

Санкт-Петербург
Есть статистически значимые различия между группами.
Среднее значение 1 ряда 0.6083916083916084
Среднее значение 2 ряда 0.6791930379746836
Разница средних =  -0.07080142958307512
P_value =  1.8595141124883944e-07

Казань
Есть статистически значимые различия между группами.
Среднее значение 1 ряда 0.596244131455399
Среднее значение 2 ряда 0.6784565916398714
Разница средних =  -0.08221246018447237
P_value =  0.0023855694044274188

Архангельск
Статистически значимых различий между группами нет.
Среднее значение 1 ряда 0.572139303482587
Сре

### 3.4

In [17]:
peak_hours = [7, 8, 9, 10, 17, 18, 19, 20]
df['flag_peak'] = df['time_view'].dt.hour.isin(peak_hours).astype(int)

In [18]:
df

,id_view,time_view,id_group,id_city,nflag_order,name_city,flag_peak
0,10457162393,2023-01-17 13:56:06.118,0,3,1,Новосибирск,0
1,10457197478,2023-01-02 11:02:58.189,0,3,0,Новосибирск,0
2,10457135372,2023-01-15 10:03:58.011,0,3,0,Новосибирск,1
3,10457162971,2023-01-02 09:39:31.283,0,3,1,Новосибирск,1
4,10457132942,2023-01-23 19:18:03.516,0,3,0,Новосибирск,1
...,...,...,...,...,...,...,...
21934,10457174004,2023-01-21 11:47:30.578,0,9,1,Воронеж,0
21935,10457130320,2023-01-24 02:09:49.169,0,9,0,Воронеж,0
21936,10457166866,2023-01-13 16:22:38.491,1,9,0,Воронеж,0
21937,10457186695,2023-01-28 02:21:49.007,0,9,1,Воронеж,0


In [19]:
df_peak = df[df['flag_peak'] == 1]
df_not_peak = df[df['flag_peak'] == 0]

In [20]:
df_peak

,id_view,time_view,id_group,id_city,nflag_order,name_city,flag_peak
2,10457135372,2023-01-15 10:03:58.011,0,3,0,Новосибирск,1
3,10457162971,2023-01-02 09:39:31.283,0,3,1,Новосибирск,1
4,10457132942,2023-01-23 19:18:03.516,0,3,0,Новосибирск,1
7,10457167469,2023-01-08 20:39:06.840,1,3,1,Новосибирск,1
9,10457162002,2023-01-17 18:45:35.647,0,3,1,Новосибирск,1
...,...,...,...,...,...,...,...
21919,10457181501,2023-01-03 20:37:20.131,1,9,0,Воронеж,1
21921,10457148587,2023-01-15 19:25:08.214,1,9,0,Воронеж,1
21922,10457178410,2023-01-23 09:46:34.139,0,9,0,Воронеж,1
21925,10457177413,2023-01-06 08:26:24.122,1,9,1,Воронеж,1


In [21]:
df_not_peak

,id_view,time_view,id_group,id_city,nflag_order,name_city,flag_peak
0,10457162393,2023-01-17 13:56:06.118,0,3,1,Новосибирск,0
1,10457197478,2023-01-02 11:02:58.189,0,3,0,Новосибирск,0
5,10457188717,2023-01-08 23:42:54.023,0,3,1,Новосибирск,0
6,10457156641,2023-01-15 06:08:14.919,1,3,1,Новосибирск,0
8,10457125370,2023-01-15 22:47:00.255,1,3,1,Новосибирск,0
...,...,...,...,...,...,...,...
21934,10457174004,2023-01-21 11:47:30.578,0,9,1,Воронеж,0
21935,10457130320,2023-01-24 02:09:49.169,0,9,0,Воронеж,0
21936,10457166866,2023-01-13 16:22:38.491,1,9,0,Воронеж,0
21937,10457186695,2023-01-28 02:21:49.007,0,9,1,Воронеж,0


In [22]:
g_a_p = df_peak[df_peak['id_group'] == 0]['nflag_order']
g_b_p = df_peak[df_peak['id_group'] == 1]['nflag_order']
s, p = test_calc(g_a_p, g_b_p)

Есть статистически значимые различия между группами.
Среднее значение 1 ряда 0.6051008968609866
Среднее значение 2 ряда 0.6770637268082818
Разница средних =  -0.07196282994729519
P_value =  1.429452278435098e-10


In [23]:
g_a_n = df_not_peak[df_not_peak['id_group'] == 0]['nflag_order']
g_b_n = df_not_peak[df_not_peak['id_group'] == 1]['nflag_order']
s, p = test_calc(g_a_n, g_b_n)

Есть статистически значимые различия между группами.
Среднее значение 1 ряда 0.6200055111600992
Среднее значение 2 ряда 0.6751420070327292
Разница средних =  -0.05513649587263003
P_value =  2.723734415858154e-12


### 3.5

На основе обновленных данных можно подтвердить вывод, что введение новой платежной механики привело к изменению поведения пользователей. В условии не сказано какая группа какую видит цену. Но если учитывать, что 0 - старая цена, 1 - новая цена, то результат теста доказывает, что поведение изменилось в положительную сторону, но только в городах Москва, Санкт-Петербург и Казань. 
Однако результат говорит лишь о том, что конверсия из просмотр в заказ выросла. Но добились этого мы путем снижения стоимости заказа. Соответственно для принятия окончательного решения об изменении механики, необходимо оценить и прочие показатели. Например цены поездки, ведь если снижение было существенным, то повышение конверсии может не перекрыть уменьшение выручки. 